In [1]:
# how different are relapse to emc92 genes?

# Predict states based on gene subset ranking

In [2]:
import json,mygene,pandas,numpy

import matplotlib,matplotlib.pyplot
matplotlib.rcParams.update({'font.size':18,'font.family':'Arial','xtick.labelsize':14,'ytick.labelsize':14,'axes.labelsize':18})
matplotlib.rcParams['pdf.fonttype']=42

In [3]:
def histogrammer(x):

    # get number of bins based on Rice's rule
    rice=int((len(x)**(1/3))*2)
    print('\t number of bins according to Rice: {}'.format(rice))

    counts,edges=numpy.histogram(x,bins=rice)
    half=(edges[1]-edges[0])/2
    centers=edges[:-1]+half

    return centers,counts

# 0. user defined variables

In [4]:
counts_file='/Volumes/GoogleDrive/My Drive/projects/MINER/shared/MINER/data/expression/MMRF_CoMMpass_IA9_E74GTF_Salmon_Gene_TPM.txt'
zscore_file='/Volumes/GoogleDrive/My Drive/projects/MINER/shared/MINER/data/expression/IA12Zscore.csv'
emc92_annotation_file='/Volumes/GoogleDrive/My Drive/projects/MINER/data/emc92.csv'
transcriptional_states_file='/Users/alomana/Google Drive File Stream/My Drive/projects/MINER/shared/MINER/results_minCorrelation_0o2_50_allFiles/transcriptional_states.json'

# 1. read data

## 1.1 read gene expression data

In [5]:
counts=pandas.read_csv(counts_file,header=0,index_col=0,sep="\t")
print(counts.shape)
counts.head()

(57997, 734)


,MMRF_2490_1_BM,MMRF_2499_1_BM,MMRF_1089_1_BM,MMRF_1778_1_BM,MMRF_1777_1_BM,MMRF_1153_1_BM,MMRF_1392_1_BM,MMRF_1325_1_BM,MMRF_1846_1_BM,MMRF_1935_1_BM,...,MMRF_1847_1_BM,MMRF_1787_1_BM,MMRF_1048_1_BM,MMRF_2437_1_BM,MMRF_1242_5_BM,MMRF_1497_1_BM,MMRF_1380_2_BM,MMRF_1700_1_BM,MMRF_1918_1_BM,MMRF_1670_1_BM
GENE_ID,,,,,,,,,,,,,,,,,,,,,
ERCC-00170,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ERCC-00168,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ERCC-00165,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ERCC-00163,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ERCC-00157,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
# zscore is not going to work because is relative ranks across patients, not within patient
zscore=pandas.read_csv(zscore_file)
print(zscore.shape)
zscore.head()

(26504, 882)


,GENE_ID,MMRF_1270_1_BM,MMRF_1037_1_BM,MMRF_2644_1_BM,MMRF_1235_1_BM,MMRF_1621_1_BM,MMRF_2313_1_BM,MMRF_1936_1_BM,MMRF_2006_1_BM,MMRF_2720_1_BM,...,MMRF_2001_1_BM,MMRF_1295_1_BM,MMRF_2721_1_BM,MMRF_1296_1_BM,MMRF_2125_1_BM,MMRF_1374_1_BM,MMRF_2525_1_BM,MMRF_1307_1_BM,MMRF_1807_1_BM,MMRF_2082_1_BM
0,ENSG00000110514,-0.387856,0.379290,0.686232,-0.659362,0.927735,0.781993,-0.561169,0.753018,0.269593,...,-0.849967,0.013019,2.210832,-1.588160,-0.784356,-0.679037,-0.878417,0.170076,0.864430,-0.743737
1,ENSG00000086015,0.608820,0.504245,-0.951228,-2.556728,1.046550,-0.182401,-0.038814,-0.111105,-0.547420,...,0.179762,0.517089,2.549893,0.006599,-1.316915,-0.301876,0.395866,1.368218,0.401868,0.714892
2,ENSG00000115808,0.618586,2.213591,2.586008,0.621943,-0.548566,0.251943,2.001565,0.195165,2.848124,...,0.127571,-0.244017,-0.281470,0.002500,1.113398,-0.289793,0.618586,0.587543,-0.827383,0.516281
3,ENSG00000204228,-0.079949,-1.403902,0.645959,0.847333,0.014213,1.134558,-2.187105,-1.422377,-0.489434,...,0.249679,0.049596,-0.812524,-0.465458,0.161633,-0.024934,0.589912,-1.130808,-0.159619,-2.561654
4,ENSG00000169740,-1.517376,-0.837227,0.438951,-0.011051,-0.182981,-0.469665,-1.179750,-0.737917,-0.836301,...,-1.078198,0.065170,-1.893471,-0.383284,0.179901,0.769384,0.184804,0.833200,-0.824710,-1.652194


## 1.2. read EMC92 genes

In [7]:
df=pandas.read_csv(emc92_annotation_file,header=None,sep=',')
print(df.shape)
df.head()

(92, 2)


,0,1
0,202728_s_at,LTBP1
1,239054_at,SFMBT1
2,208942_s_at,SEC62
3,208747_s_at,C1S
4,202542_s_at,AIMP1


In [8]:
genes=df[1].to_list()
print(genes,len(genes))

['LTBP1', 'SFMBT1', 'SEC62', 'C1S', 'AIMP1', 'ZBTB25', 'ACVR2A', 'S100A6', 'ITGA6', 'HMGB3', 'BCS1L', 'LOC100271836', 'PPP2R1B', 'PCDHB7', 'DYNLRB2', 'IL7R', 'DNAJB9', 'RAB2A', 'FGFR3', 'TSPAN16', 'DUX4', 'FAM49A', 'ROBO3', 'SUN1', 'TARBP1', 'TMEM97', 'EST/BX647543', 'STAT1', 'SLC17A5', 'DHRS9', 'MAGEA6', '---', 'ATPBD4', 'GRB14', 'GABRA4', 'NOP56', 'ESPL1', 'C15orf38', 'CCRL1', 'MARCKS', 'EST/BE568408', 'EHBP1L1', 'ST13', 'C18orf10', 'TOP2A', 'PFKL', 'ATP6V0E1', 'SAR1B', 'CDH22', 'RPS28', 'MRPL41', 'RPS4X', 'SLC30A7', 'ARPC5', 'FANCF', 'TUBB', 'TRAM1', 'ITM2B', 'MCM2', 'TUBB', 'NPC2', 'HMGN5', 'DTL', 'ANGEL2', 'ZNF252', 'NCAPG', 'BIRC5', 'SEPT11', 'FTL', 'HNRNPK', 'SPATS2L', 'PGM2', 'GGPS1', 'DONSON', 'KIF4A', 'AK2', 'FANCI', 'POLQ', 'C11orf85', 'MCM6', 'CENPE', 'NUF2', 'LBR', 'RPS11', 'SYF2', 'MCM3', 'ZWINT', 'NCUBE1', 'DARS2', 'ARL8B', 'DHFR', 'SPAG5'] 92


In [9]:
genes[genes.index('EST/BX647543')]='KCNJ5'
genes[genes.index('EST/BE568408')]='BBOX1-AS1'
genes[genes.index('LOC100271836')]='SMG1P3'
genes[genes.index('ZNF252')]='ZNF252P'
genes.remove('---')
print(genes,len(genes))
print(len(list(set(genes))))
set([x for x in genes if genes.count(x) > 1])

['LTBP1', 'SFMBT1', 'SEC62', 'C1S', 'AIMP1', 'ZBTB25', 'ACVR2A', 'S100A6', 'ITGA6', 'HMGB3', 'BCS1L', 'SMG1P3', 'PPP2R1B', 'PCDHB7', 'DYNLRB2', 'IL7R', 'DNAJB9', 'RAB2A', 'FGFR3', 'TSPAN16', 'DUX4', 'FAM49A', 'ROBO3', 'SUN1', 'TARBP1', 'TMEM97', 'KCNJ5', 'STAT1', 'SLC17A5', 'DHRS9', 'MAGEA6', 'ATPBD4', 'GRB14', 'GABRA4', 'NOP56', 'ESPL1', 'C15orf38', 'CCRL1', 'MARCKS', 'BBOX1-AS1', 'EHBP1L1', 'ST13', 'C18orf10', 'TOP2A', 'PFKL', 'ATP6V0E1', 'SAR1B', 'CDH22', 'RPS28', 'MRPL41', 'RPS4X', 'SLC30A7', 'ARPC5', 'FANCF', 'TUBB', 'TRAM1', 'ITM2B', 'MCM2', 'TUBB', 'NPC2', 'HMGN5', 'DTL', 'ANGEL2', 'ZNF252P', 'NCAPG', 'BIRC5', 'SEPT11', 'FTL', 'HNRNPK', 'SPATS2L', 'PGM2', 'GGPS1', 'DONSON', 'KIF4A', 'AK2', 'FANCI', 'POLQ', 'C11orf85', 'MCM6', 'CENPE', 'NUF2', 'LBR', 'RPS11', 'SYF2', 'MCM3', 'ZWINT', 'NCUBE1', 'DARS2', 'ARL8B', 'DHFR', 'SPAG5'] 91
90


{'TUBB'}

In [10]:
emc92_ensembl_conversion={}
for gene in genes:
    emc92_ensembl_conversion[gene]=[]
    
    result=mygene.MyGeneInfo().query(gene,scopes="symbol",fields=["ensembl"],species="human",verbose=True)
    
    for hit in result['hits']:
        score=hit['_score']
        if score > 1:
            if 'ensembl' in hit:
                if type(hit['ensembl']) == dict:
                    elements=[hit['ensembl']]
                elif type(hit['ensembl']) == list:
                    elements=hit['ensembl']
                else:
                    raise Exception("Error: variable type not recognized.")
                for element in elements:
                    ensembl_ID=element['gene']
                    type_of_gene=element['type_of_gene']
                    emc92_ensembl_conversion[gene].append((ensembl_ID,type_of_gene,score))
            
print(len(emc92_ensembl_conversion.keys()))

90


In [11]:
# map emc92 ensembl ids to counts
found_IDs=[]
for element in emc92_ensembl_conversion:
    found=False
    for hit in emc92_ensembl_conversion[element]:
        ensembl_ID=hit[0]  
        if ensembl_ID in counts.index and found == False:
            found_IDs.append(ensembl_ID)
            found=True
print(len(found_IDs))
print(len(list(set(found_IDs))))

90
90


## 1.3. obtain model state classification

In [12]:
# read data
with open(transcriptional_states_file) as json_file:
    states = json.load(json_file)

In [13]:
# remove peripheral blood samples
bm_states={}
for state_ID in states:
    bm=[element for element in states[state_ID] if element.split('_')[-1] == 'BM']
    bm_states[state_ID]=bm

In [14]:
# get a list of BM patients
all_patients=[]
for state_ID in bm_states:
    for patient in bm_states[state_ID]:
        if patient not in all_patients:
            all_patients.append(patient)
print(len(all_patients))

821


In [15]:
# intersect with count headers
intersect=list(set(counts.columns) & set(all_patients))

# 2. prepare final data structures

## 2.1. expression

In [16]:
print(len(intersect))
df=counts[intersect]
df=df.loc[found_IDs]
print(df.shape)
df.head()

683
(90, 683)


,MMRF_1460_1_BM,MMRF_1786_1_BM,MMRF_2306_1_BM,MMRF_1300_1_BM,MMRF_1750_1_BM,MMRF_1338_1_BM,MMRF_1846_1_BM,MMRF_2204_1_BM,MMRF_1277_1_BM,MMRF_1795_1_BM,...,MMRF_1825_1_BM,MMRF_2115_1_BM,MMRF_2273_1_BM,MMRF_1977_1_BM,MMRF_2475_1_BM,MMRF_2173_1_BM,MMRF_1324_1_BM,MMRF_2076_1_BM,MMRF_1895_1_BM,MMRF_2091_1_BM
GENE_ID,,,,,,,,,,,,,,,,,,,,,
ENSG00000049323,2.12091,3.41460,3.488690,2.430740,3.909170,4.816720,5.08434,1.109060,4.891980,11.201100,...,0.991506,2.738000,7.70579,9.78660,2.399990,5.344160,1.60191,1.10934,3.83956,5.96672
ENSG00000163935,4.51837,5.38089,0.657739,2.690300,2.119080,4.147420,10.60450,1.864870,5.259220,6.549500,...,3.410140,3.103870,3.61085,9.13069,3.983870,2.616340,7.02224,8.08953,6.17160,2.55872
ENSG00000008952,50.04810,44.45520,39.722500,102.446000,69.870400,102.951000,66.27540,52.590300,90.554100,81.479500,...,60.105300,56.364600,82.86390,130.16800,50.583600,68.837400,96.69380,87.78140,177.19300,76.09440
ENSG00000182326,1.63962,1.17465,0.767181,0.495743,0.304173,0.326874,0.37708,0.303516,0.360946,0.598107,...,1.541640,0.540504,1.41893,1.92959,0.897386,0.938797,1.15475,1.28573,2.06895,0.39991
ENSG00000164022,24.96130,17.29220,13.563000,22.382400,8.948560,14.143200,21.60700,13.401100,26.312000,17.676800,...,26.016600,0.301967,29.10140,35.46460,11.149700,19.325800,19.57090,35.80800,25.41450,26.53920


In [ ]:
# consider normalize by gene length if necessary and then rank

## 2.2. states

In [17]:
states={}
for state in bm_states:
    states[state]=[]
    for patient in bm_states[state]:
        if patient in intersect:
            states[state].append(patient)

In [18]:
sizes=[]
for state in bm_states:
    print(state,len(bm_states[state]),len(states[state]))
    sizes.append(len(states[state]))
sizes.sort(reverse=True)
print(sizes)

24 2 2
20 41 37
21 13 12
22 13 11
23 29 26
1 96 64
0 104 52
3 92 90
2 30 19
5 20 18
4 31 29
7 36 33
6 6 6
9 6 5
8 17 16
11 54 52
10 10 10
13 18 17
12 23 18
15 43 40
14 22 15
17 30 28
16 62 61
19 4 4
18 19 18
[90, 64, 61, 52, 52, 40, 37, 33, 29, 28, 26, 19, 18, 18, 18, 17, 16, 15, 12, 11, 10, 6, 5, 4, 2]


# 3. find matrices of pair differential occurrence

In [19]:
# define the n most entropic pairs, for each state. Then get the call using proportions on information for likelihood. Likelihood might not be needed.

In [20]:
# this is going to be state-specific. At least 5 patients please

In [21]:
# make a confusion matrix of true and predictederk[]


# 3. find biomarkers: state-specific perfect pair sets

# 4. how useful are sets of random genes?